In [1]:
import matplotlib as mpl
mpl.use("pgf")
pgf_with_pdflatex = {
    "pgf.texsystem": "xelatex",
    "font.family": "serif", # use serif/main font for text elements
    "text.usetex": True,    # use inline math for ticks
    "pgf.rcfonts": False, 
    "pgf.preamble": [
        #r"\usepackage{amsmath}",
        #r"\usepackage{xunicode}",
        r"\usepackage{fontspec}",
        r"\usepackage{xltxtra}",
        r"\setmainfont{Minion-Pro_Regular.ttf}[BoldFont = Minion-Pro-Bold.ttf, ItalicFont = Minion-Pro-Italic.ttf, BoldItalicFont = Minion-Pro-Bold-Italic.ttf]"
         ]
}
mpl.rcParams.update(pgf_with_pdflatex)

In [2]:
import matplotlib.pyplot as plt
import numpy as np

In [3]:
e=1.602*10**-19 #As
eps_0=8.85*10**-12 #As/Vm
eps_r=3.5
carrierType=1
offsetCathode=1 #1=true -1=false
V=0
E=V/(200*10**-9)#V/m; 200nm organics
wf_m=-4.2*e #J
wf_o=-2.8*e #J
Delta=wf_m-wf_o
def points(x,dim):
    return x/dim
initpos=0
x_scale=10**9
filename="test2.pdf"
x_min=-7
metalContact=False
def V_0(x, wf, b):
    if b*offsetCathode==-3 or abs(b)==2:
        return [carrierType*wf if a>=0 else 0.0 for a in x]
    if b*offsetCathode==3:
        return [carrierType*wf+V*e if a>=0 else 0.0 for a in x]
def V_1(x, wf, sigma=0.1*e, points=points, mu=0):
    z=np.random.normal(mu, sigma, points)
    return [carrierType*wf+c if a>=0 else 0.0 for a,c in zip(x,z)]
def V_2(x, b):
    if b==0:
        return 0
    if b==-1:
        return [carrierType*(e**2)/(16*np.pi*eps_0*eps_r*(a-x[0])) if a>=0 else 0.0 for a in x]
    if b==1:
        return [carrierType*(e**2)/(16*np.pi*eps_0*eps_r*(a-x[len(x)-1])) if a>=0 else 0.0 for a in x]
def V_3(x, offset, E=E):
    return [carrierType*e*E*(a-offset) if a>=0 else 0.0 for a in x]

In [4]:
class Material:
    iD=0
    def __init__(self, thickness, CBLike, name="Unknown Material", x=0, y=0, vacY=0, dim=10*, metallic=False):
        self.name=name
        self.thickness = thickness
        self.CBLike = CBLike
        self.x = x
        self.dim = 
        self.y = y
        self.vacY=vacY
        self.metallic=metallic
        self.id = Material.iD+1
        Material.iD=Material.iD+1
    def __repr__(self):
        return "{}: ({:3.0f} nm)".format(self.name,self.thickness*10**9)
    def __str__(self):
        return self.name
    def name():
        return self.name
    

In [5]:
class Organic(Material):
    def __init__(self, thickness, CBLike, VLLike, name="Unknown Material", x=0, y=0, VLDevia=0.05, CBDevia=0.05, eps_r=3.5, metallic=False, y_2=0, sigma=0.001, polarity=0):
        Material.__init__(self, thickness, CBLike, name=name, x=x, y=y, metallic=False)
        self.VLLike = VLLike
        self.VLDevia=VLDevia
        self.CBDevia=CBDevia
        self.eps_r=eps_r
        self.y_2= y_2
        self.sigma= sigma
        self.polarity=polarity
    

In [6]:
class Metal(Material):
    def __init__(self, thickness, CBLike, name="Unknown Material", x=0, y=0):
        Material.__init__(self, thickness, CBLike, name=name, x=x, y=y, metallic=True)

In [7]:
Materials=[
Metal(150*10**-9, -4.8*e, name="ITO"),
 Metal(20*10**-9, -5.0*e, name="PEDOT:PSS"),
 Organic(20*10**-9, -2.3*e, -5.2*e, name="poly-TPD", sigma=0.01*e),
 Organic(75*10**-9, -2.85*e, -5.6*e, name="Alq3", sigma=0.1*e),
 Metal(10*10**-9, -2.83*e, name="Ca"),
 Metal(100*10**-9, -4.2*e, name="Al")]

In [8]:
curPos=initpos
for m in Materials:
    nexPos= curPos+m.thickness
    m.x=np.linspace(curPos,nexPos,points)
    curPos= nexPos

In [9]:
A=[m.metallic for m in Materials]
hil=A.index(False)
eil=len(A) - A[::-1].index(False) - 1
B=[-3]+[-2]*(hil-1)+[-1]+(eil-hil-1)*[0]+[1]+(len(A)-2-eil)*[2]+[3]
C=["red","blue","green","cyan","yellow","magenta"]

In [10]:
ax2 = plt.subplot()
ax2.set_xlabel("\\textbf{Depth ($\\mathrm{nm}$)}")
ax2.set_ylabel("\\textbf{Potential ($\\mathrm{eV}$)}")
#ax2.set_xscale("log", basex=10, subsx=[2,3,4,5,6,7,8,9])
#ax2.set_yscale("log", basex=10, subsy=[2,3,4,5,6,7,8,9])
#ax2.set_ylim(10**-3, 10**0)
#ax2.grid(which="major")
#ax2.grid(which="minor", ls=":", alpha=0.5)


In [11]:
for m,b in zip(Materials, B):
    if b*offsetCathode==3:
        a=m.CBLike
    if b*-offsetCathode==3:
        c=m.CBLike
    if not m.metallic:
        m.y=np.asarray(V_1(m.x,m.CBLike, sigma=m.sigma))+np.asarray(V_2(m.x, b))+np.asarray(V_3(m.x, m.x[0]))+np.asarray(V_3(m.x,m.x[0], E=m.polarity))
        m.y_2=np.asarray(V_1(m.x,m.VLLike, sigma=m.sigma))+np.asarray(V_2(m.x, b))+np.asarray(V_3(m.x, m.x[0]))+np.asarray(V_3(m.x,m.x[0], E=m.polarity))
        m.vacY=np.asarray(V_2(m.x, b))+np.asarray(V_3(m.x, m.x[0]))+np.asarray(V_3(m.x,m.x[0], E=m.polarity))
        plt.plot(np.asarray(m.x)*x_scale,m.y*e**-1, color=C[b-1], label=m.name)
        plt.plot(np.asarray(m.x)*x_scale,m.y_2*e**-1, color=C[b-1])
        plt.fill_between(np.asarray(m.x)*x_scale,x_min,m.y_2*e**-1, facecolor=C[b-1],interpolate=True, alpha=0.3)
        plt.fill_between(np.asarray(m.x)*x_scale,m.y*e**-1,m.y_2*e**-1, facecolor=C[b-1],interpolate=True, alpha=0.1)
        plt.text(m.x[(len(m.x)-1)//2]*x_scale,(m.y_2[(len(m.x)-1)//2]+m.y[(len(m.x)-1)//2])/2*e**-1,m.name, ha="center", va="center")
        plt.plot(np.asarray(m.x)*x_scale,m.vacY*e**-1, "k-")
for m,b in zip(Materials, B):
    if m.metallic and abs(b)==2 and metalContact:
        if b==-2:
            m.y=V_0(m.x,c, 3*-offsetCathode)
        if b==2:
            m.y=V_0(m.x,a, 3*offsetCathode)
        m.vacY=[d-m.CBLike for d in m.y]
        plt.plot(np.asarray(m.x)*x_scale,np.asarray(m.y)*e**-1, color=C[b-1], label=m.name)
        plt.fill_between(np.asarray(m.x)*x_scale,x_min,np.asarray(m.y)*e**-1, interpolate=True, facecolor=C[b-1], alpha=0.3)
        plt.text(m.x[(len(m.x)-1)//2]*x_scale,(m.y[0]+x_min*e)/2*e**-1,m.name, ha="center", va="center")
        plt.plot(np.asarray(m.x)*x_scale,np.asarray(m.vacY)*e**-1, "k-",label="Vacuum")
    elif m.metallic:
        m.y=V_0(m.x,m.CBLike, b)
        m.vacY=V_0(m.x,0,b)
        plt.plot(np.asarray(m.x)*x_scale,np.asarray(m.y)*e**-1, color=C[b-1], label=m.name)
        plt.fill_between(np.asarray(m.x)*x_scale,x_min,np.asarray(m.y)*e**-1,interpolate=True, facecolor=C[b-1], alpha=0.3)
        plt.text(m.x[(len(m.x)-1)//2]*x_scale,(m.y[0]+x_min*e)/2*e**-1,m.name, ha="center", va="center")
        plt.plot(np.asarray(m.x)*x_scale,np.asarray(m.vacY)*e**-1, "k-")

/home/tassilo/Programme/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:29: RuntimeWarning: divide by zero encountered in double_scalars
/home/tassilo/Programme/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:31: RuntimeWarning: divide by zero encountered in double_scalars


In [12]:
ax2.set_title("\\textbf{Energy States Alignment}", fontsize=16)
ax2.set_ylim(x_min, V+1)
#handles, labels=ax2.get_legend_handles_labels()
#del handles[-2]
#del labels[-2]
#ax2.legend(handles, labels, loc=2, numpoints=1, fontsize=8)
plt.tight_layout()
plt.savefig(filename)